# Anotação Automática de Datasets usando Múltiplas LLMs

## Análise de Consenso e Validação de Metodologia
---

### Objetivo

Este notebook implementa uma metodologia para reduzir o custo humano na anotação de datasets através do uso de múltiplas LLMs e análise de consenso.

### Metodologia

1. **Anotação Múltipla**: 5 LLMs diferentes anotam cada instância
2. **Consenso Interno**: Cada LLM anota múltiplas vezes a mesma instância
3. **Análise de Consenso**: Calculamos métricas de concordância entre LLMs
4. **Validação de Parâmetros**: Testamos se variações nos parâmetros afetam os resultados
5. **Estratégias de Resolução**: Definimos como tratar casos sem consenso claro

## 1. Setup e Imports

In [ ]:
# Imports necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Imports dos módulos customizados
from llm_annotator import LLMAnnotator
from consensus_analyzer import ConsensusAnalyzer
from visualizer import ConsensusVisualizer
from config import LLM_CONFIGS, EXPERIMENT_CONFIG

# Configuração de visualização
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✓ Imports realizados com sucesso!")

## 2. Configuração de API Keys

**IMPORTANTE:** Configure suas API keys aqui ou use um arquivo .env

In [ ]:
# Opção 1: Configuração direta (NÃO COMMITAR PARA GIT!)
api_keys = {
    "openai": "sua-api-key-aqui",
    "anthropic": "sua-api-key-aqui",
    "google": "sua-api-key-aqui",
}

# Opção 2: Carregar de arquivo .env (RECOMENDADO)
# from dotenv import load_dotenv
# import os
# load_dotenv()
# api_keys = {
#     "openai": os.getenv("OPENAI_API_KEY"),
#     "anthropic": os.getenv("ANTHROPIC_API_KEY"),
#     "google": os.getenv("GOOGLE_API_KEY"),
# }

print("✓ API Keys configuradas")

## 3. Carregar Dataset

Carregue seu dataset aqui. O dataset deve ter pelo menos:
- Uma coluna com textos a serem anotados
- (Opcional) Uma coluna com labels verdadeiros para validação

In [ ]:
# Exemplo: Carregar dataset
# df_dataset = pd.read_csv('seu_dataset.csv')

# Para demonstração, vamos criar um dataset de exemplo
example_texts = [
    "Este produto é excelente! Recomendo muito.",
    "Péssima qualidade, não funciona como esperado.",
    "O produto é ok, nada de especial.",
    "Maravilhoso! Superou minhas expectativas.",
    "Horrível, totalmente decepcionado.",
]

# Categorias disponíveis
categories = ["Positivo", "Negativo", "Neutro"]

print(f"Dataset carregado: {len(example_texts)} textos")
print(f"Categorias: {categories}")

## 4. Configurar Modelos LLM

Selecione quais modelos você deseja usar para anotação

In [ ]:
# Modelos disponíveis (ajuste conforme suas API keys)
selected_models = [
    "gpt-4-turbo",
    "gpt-3.5-turbo",
    "claude-3-opus",
    "claude-3-sonnet",
    "gemini-pro",
]

print(f"Modelos selecionados: {len(selected_models)}")
for model in selected_models:
    print(f"  - {model}")

## 5. Inicializar Anotador

In [ ]:
# Inicializar o anotador
annotator = LLMAnnotator(
    models=selected_models,
    categories=categories,
    api_keys=api_keys,
    cache_dir="./cache",
    results_dir="./results"
)

print("\n✓ Anotador inicializado e pronto para uso!")

## 6. Executar Anotação

### 6.1 Anotação com Parâmetros Padrão

In [ ]:
# Anotar dataset
df_annotations = annotator.annotate_dataset(
    texts=example_texts,
    num_repetitions=3,  # Cada LLM anota 3 vezes (OBS-2)
    test_param_variations=False,  # Mudar para True para testar variações
)

# Visualizar primeiras linhas
print("\n📊 Primeiras anotações:")
display(df_annotations.head())

### 6.2 (Opcional) Testar Variações de Parâmetros

Investiga se mudanças nos parâmetros das LLMs afetam significativamente os resultados ("LLM hacking")

In [ ]:
# Descomentar para testar variações de parâmetros
# df_annotations_params = annotator.annotate_dataset(
#     texts=example_texts,
#     num_repetitions=3,
#     test_param_variations=True,  # Testa diferentes temperaturas, top_p, etc.
# )
#
# print("\n📊 Anotações com variações de parâmetros:")
# display(df_annotations_params.head())

## 7. Calcular Consenso

Análise de consenso entre as LLMs e dentro de cada LLM

In [ ]:
# Calcular métricas de consenso
df_with_consensus = annotator.calculate_consensus(df_annotations)

# Visualizar resultados com consenso
print("\n📊 Anotações com métricas de consenso:")
display(df_with_consensus[[
    'text',
    'most_common_annotation',
    'consensus_score',
    'consensus_level',
    'unique_annotations',
    'is_problematic'
]].head())

## 8. Análise Detalhada de Consenso

### 8.1 Métricas de Distância e Concordância

In [ ]:
# Inicializar analisador
analyzer = ConsensusAnalyzer(categories=categories)

# Coletar colunas de consenso
consensus_cols = [col for col in df_with_consensus.columns if '_consensus' in col and '_score' not in col]

# Gerar relatório completo
report = analyzer.generate_consensus_report(
    df=df_with_consensus,
    annotator_cols=consensus_cols,
    output_dir="./results"
)

print("\n✓ Relatório de consenso gerado!")

### 8.2 Visualizar Métricas

In [ ]:
# Exibir métricas calculadas
print("\n📊 MÉTRICAS DE DISTÂNCIA E CONCORDÂNCIA")
print("="*60)

for metric, value in report['distance_metrics'].items():
    if value is not None:
        print(f"{metric:30s}: {value:8.4f}")

# Interpretação das métricas
print("\n💡 INTERPRETAÇÃO:")
print("-" * 60)

if 'mean_cohen_kappa' in report['distance_metrics']:
    kappa = report['distance_metrics']['mean_cohen_kappa']
    if kappa > 0.8:
        print("✓ Cohen's Kappa > 0.8: Concordância EXCELENTE")
    elif kappa > 0.6:
        print("✓ Cohen's Kappa > 0.6: Concordância BOA")
    elif kappa > 0.4:
        print("⚠ Cohen's Kappa > 0.4: Concordância MODERADA")
    else:
        print("⚠ Cohen's Kappa < 0.4: Concordância FRACA")

### 8.3 Matriz de Concordância Par a Par

In [ ]:
# Visualizar matriz de concordância
print("\n📊 Matriz de Concordância Par a Par:")
display(report['pairwise_agreement'])

print("\n📊 Matriz de Cohen's Kappa Par a Par:")
display(report['pairwise_kappa'])

## 9. Análise de Instâncias Problemáticas

Identificar casos onde há discordância significativa entre as LLMs

In [ ]:
# Filtrar instâncias problemáticas
problematic = df_with_consensus[df_with_consensus['is_problematic'] == True]

print(f"\n⚠️ INSTÂNCIAS PROBLEMÁTICAS: {len(problematic)}")
print("="*80)

if len(problematic) > 0:
    for idx, row in problematic.iterrows():
        print(f"\nTexto {idx}:")
        print(f"  {row['text'][:100]}...")
        print(f"  Anotações: {row['all_annotations']}")
        print(f"  Consenso: {row['consensus_score']:.2%}")
        print(f"  Categoria mais comum: {row['most_common_annotation']}")
        print("-" * 80)
else:
    print("✓ Nenhuma instância problemática encontrada!")

### 9.1 Estratégias para Resolver Casos Problemáticos

Aqui implementamos diferentes estratégias para lidar com casos sem consenso claro

In [ ]:
def resolve_conflicts(df, strategy='majority_vote', threshold=0.6):
    """
    Resolve conflitos usando diferentes estratégias
    
    Estratégias:
    - 'majority_vote': Usa voto majoritário simples
    - 'unanimous_only': Aceita apenas casos com 100% consenso
    - 'threshold': Aceita apenas se consenso >= threshold
    - 'flag_for_review': Marca para revisão humana
    - 'remove': Remove instâncias problemáticas
    """
    df = df.copy()
    
    if strategy == 'majority_vote':
        df['final_annotation'] = df['most_common_annotation']
        df['needs_review'] = False
        
    elif strategy == 'unanimous_only':
        df['final_annotation'] = df.apply(
            lambda row: row['most_common_annotation'] if row['consensus_score'] == 1.0 else None,
            axis=1
        )
        df['needs_review'] = df['final_annotation'].isna()
        
    elif strategy == 'threshold':
        df['final_annotation'] = df.apply(
            lambda row: row['most_common_annotation'] if row['consensus_score'] >= threshold else None,
            axis=1
        )
        df['needs_review'] = df['final_annotation'].isna()
        
    elif strategy == 'flag_for_review':
        df['final_annotation'] = df['most_common_annotation']
        df['needs_review'] = df['is_problematic'] | (df['consensus_score'] < threshold)
        
    elif strategy == 'remove':
        df = df[~df['is_problematic'] & (df['consensus_score'] >= threshold)].copy()
        df['final_annotation'] = df['most_common_annotation']
        df['needs_review'] = False
    
    return df

# Testar diferentes estratégias
strategies = ['majority_vote', 'threshold', 'flag_for_review']

print("\n📊 COMPARAÇÃO DE ESTRATÉGIAS DE RESOLUÇÃO")
print("="*80)

for strategy in strategies:
    df_resolved = resolve_conflicts(df_with_consensus, strategy=strategy, threshold=0.6)
    
    print(f"\nEstratégia: {strategy}")
    print(f"  Total de instâncias: {len(df_resolved)}")
    print(f"  Anotações finais: {df_resolved['final_annotation'].notna().sum()}")
    print(f"  Necessitam revisão: {df_resolved['needs_review'].sum()}")
    print("-" * 80)

## 10. Visualizações

### 10.1 Gerar Todas as Visualizações

In [ ]:
# Inicializar visualizador
visualizer = ConsensusVisualizer(output_dir="./results/figures")

# Gerar visualizações
print("Gerando visualizações...\n")

# 1. Heatmap de concordância
visualizer.plot_agreement_heatmap(
    report['pairwise_agreement'],
    title="Concordância entre Modelos LLM"
)

# 2. Distribuição de consenso
visualizer.plot_consensus_distribution(df_with_consensus)

# 3. Matriz de confusão
if 'disagreement_patterns' in report:
    visualizer.plot_confusion_matrix(
        report['disagreement_patterns']['confusion_matrix']
    )

# 4. Comparação entre modelos
visualizer.plot_model_comparison(
    df_with_consensus,
    models=selected_models
)

# 5. Dashboard interativo
visualizer.create_interactive_dashboard(
    df_with_consensus,
    report
)

print("\n✓ Todas as visualizações foram geradas!")
print("📁 Arquivos salvos em: ./results/figures/")

### 10.2 Visualizar no Notebook

In [ ]:
from IPython.display import Image

# Exibir algumas visualizações inline
print("📊 Heatmap de Concordância:")
display(Image(filename='./results/figures/agreement_heatmap.png'))

print("\n📊 Distribuição de Consenso:")
display(Image(filename='./results/figures/consensus_distribution.png'))

## 11. Análise de Impacto de Parâmetros

Se você testou variações de parâmetros, analise o impacto aqui

In [ ]:
# Descomentar se você testou variações de parâmetros
# for model in selected_models:
#     param_cols = [col for col in df_annotations_params.columns if col.startswith(f"{model}_param_var")]
#     if param_cols:
#         visualizer.plot_parameter_impact(
#             df_annotations_params,
#             model=model
#         )

## 12. Sumário e Recomendações

### 12.1 Estatísticas Finais

In [ ]:
print("\n" + "="*80)
print(" " * 25 + "SUMÁRIO FINAL")
print("="*80)

# Estatísticas gerais
print("\n📊 ESTATÍSTICAS GERAIS:")
print("-" * 80)
print(f"Total de instâncias anotadas: {len(df_with_consensus)}")
print(f"Modelos utilizados: {len(selected_models)}")
print(f"Repetições por modelo: {EXPERIMENT_CONFIG['num_repetitions_per_llm']}")

# Consenso
print("\n📊 DISTRIBUIÇÃO DE CONSENSO:")
print("-" * 80)
print(f"Alto consenso (≥80%): {(df_with_consensus['consensus_score'] >= 0.8).sum()} ({(df_with_consensus['consensus_score'] >= 0.8).sum() / len(df_with_consensus) * 100:.1f}%)")
print(f"Médio consenso (60-80%): {((df_with_consensus['consensus_score'] >= 0.6) & (df_with_consensus['consensus_score'] < 0.8)).sum()} ({((df_with_consensus['consensus_score'] >= 0.6) & (df_with_consensus['consensus_score'] < 0.8)).sum() / len(df_with_consensus) * 100:.1f}%)")
print(f"Baixo consenso (<60%): {(df_with_consensus['consensus_score'] < 0.6).sum()} ({(df_with_consensus['consensus_score'] < 0.6).sum() / len(df_with_consensus) * 100:.1f}%)")

# Casos problemáticos
print("\n⚠️  CASOS PROBLEMÁTICOS:")
print("-" * 80)
print(f"Total: {df_with_consensus['is_problematic'].sum()} ({df_with_consensus['is_problematic'].sum() / len(df_with_consensus) * 100:.1f}%)")

# Métricas de concordância
print("\n📊 MÉTRICAS DE CONCORDÂNCIA:")
print("-" * 80)
if 'mean_cohen_kappa' in report['distance_metrics']:
    print(f"Cohen's Kappa médio: {report['distance_metrics']['mean_cohen_kappa']:.4f}")
if 'fleiss_kappa' in report['distance_metrics'] and report['distance_metrics']['fleiss_kappa']:
    print(f"Fleiss' Kappa: {report['distance_metrics']['fleiss_kappa']:.4f}")
if 'mean_jaccard_similarity' in report['distance_metrics']:
    print(f"Jaccard Similarity médio: {report['distance_metrics']['mean_jaccard_similarity']:.4f}")

print("\n" + "="*80)

### 12.2 Recomendações

In [ ]:
print("\n💡 RECOMENDAÇÕES:")
print("="*80)

# Baseado no consenso médio
consensus_mean = df_with_consensus['consensus_score'].mean()

if consensus_mean >= 0.8:
    print("\n✓ EXCELENTE! O consenso entre as LLMs é muito alto.")
    print("  Recomendação: As anotações automáticas são confiáveis para a maioria dos casos.")
    print("  Sugestão: Revisar apenas os casos marcados como problemáticos.")
elif consensus_mean >= 0.6:
    print("\n⚠ BOM. O consenso é satisfatório, mas há espaço para melhoria.")
    print("  Recomendação: Considere revisar casos com consenso < 70%.")
    print("  Sugestão: Testar prompts alternativos ou adicionar exemplos (few-shot).")
else:
    print("\n⚠️ ATENÇÃO. O consenso entre as LLMs é baixo.")
    print("  Recomendação: Revisão humana extensiva é necessária.")
    print("  Sugestão: Revisar as categorias e melhorar os prompts.")

# Baseado em casos problemáticos
problematic_ratio = df_with_consensus['is_problematic'].sum() / len(df_with_consensus)

if problematic_ratio > 0.2:
    print("\n⚠️ ATENÇÃO: Mais de 20% dos casos são problemáticos.")
    print("  Sugestão: Considere refinar as categorias ou usar prompts mais específicos.")

# Baseado em Cohen's Kappa
if 'mean_cohen_kappa' in report['distance_metrics']:
    kappa = report['distance_metrics']['mean_cohen_kappa']
    if kappa < 0.6:
        print("\n⚠️ Cohen's Kappa indica concordância moderada ou fraca.")
        print("  Sugestão: Considere:")
        print("    1. Adicionar exemplos (few-shot learning)")
        print("    2. Melhorar definições de categorias")
        print("    3. Usar modelos mais avançados")

print("\n" + "="*80)

## 13. Exportar Resultados

Exportar todos os resultados para análise posterior

In [ ]:
# Criar diretório de resultados finais
final_results_dir = Path("./results/final")
final_results_dir.mkdir(parents=True, exist_ok=True)

# Exportar dataset anotado completo
df_with_consensus.to_csv(
    final_results_dir / "annotated_dataset_complete.csv",
    index=False,
    encoding='utf-8'
)

# Exportar apenas anotações de alta confiança
high_confidence = df_with_consensus[df_with_consensus['consensus_score'] >= 0.8].copy()
high_confidence[['text', 'final_annotation', 'consensus_score']].to_csv(
    final_results_dir / "high_confidence_annotations.csv",
    index=False,
    encoding='utf-8'
)

# Exportar casos para revisão
needs_review = df_with_consensus[
    (df_with_consensus['consensus_score'] < 0.6) | (df_with_consensus['is_problematic'] == True)
].copy()
needs_review.to_csv(
    final_results_dir / "needs_human_review.csv",
    index=False,
    encoding='utf-8'
)

# Exportar sumário em JSON
import json

summary = {
    "experiment_config": EXPERIMENT_CONFIG,
    "models_used": selected_models,
    "categories": categories,
    "total_instances": len(df_with_consensus),
    "consensus_statistics": {
        "mean": float(df_with_consensus['consensus_score'].mean()),
        "median": float(df_with_consensus['consensus_score'].median()),
        "std": float(df_with_consensus['consensus_score'].std()),
    },
    "distribution": {
        "high_consensus": int((df_with_consensus['consensus_score'] >= 0.8).sum()),
        "medium_consensus": int(((df_with_consensus['consensus_score'] >= 0.6) & (df_with_consensus['consensus_score'] < 0.8)).sum()),
        "low_consensus": int((df_with_consensus['consensus_score'] < 0.6).sum()),
    },
    "problematic_instances": int(df_with_consensus['is_problematic'].sum()),
    "distance_metrics": {k: float(v) if v is not None else None for k, v in report['distance_metrics'].items()},
}

with open(final_results_dir / "experiment_summary.json", 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print("\n✓ Resultados exportados com sucesso!")
print(f"📁 Diretório: {final_results_dir}")
print("\nArquivos gerados:")
print("  - annotated_dataset_complete.csv: Dataset completo com todas as anotações")
print("  - high_confidence_annotations.csv: Apenas anotações de alta confiança")
print("  - needs_human_review.csv: Casos que precisam de revisão humana")
print("  - experiment_summary.json: Sumário do experimento")

## 14. Conclusões e Próximos Passos

### Principais Achados:

1. **Consenso entre LLMs**: [A ser preenchido após análise]
2. **Impacto de parâmetros**: [A ser preenchido após análise]
3. **Casos problemáticos**: [A ser preenchido após análise]
4. **Economia de custo**: [Calcular percentual de instâncias que não precisam revisão humana]

### Próximos Passos:

1. [ ] Validar anotações automáticas com ground truth (se disponível)
2. [ ] Testar com diferentes prompts (few-shot, Chain-of-Thought)
3. [ ] Escalar para datasets maiores
4. [ ] Implementar sistema de revisão humana para casos problemáticos
5. [ ] Documentar custos de API e tempo de execução
6. [ ] Apresentar resultados para orientador

### Material para Discussão com Orientador:

- Este notebook completo com análises
- Visualizações geradas em `./results/figures/`
- Dashboard interativo em `./results/figures/interactive_dashboard.html`
- Sumário do experimento em `./results/final/experiment_summary.json`
- Matriz de concordância e métricas de distância